In [1]:
import DataConnection as Dc
from collections import defaultdict
import time
import datetime
import operator
import csv



pgo = Dc.PGConnection('CRASHDATA', 'shostetter')  #Database Connection
dbo = Dc.DBConnection('dotgissql01', 'gisgrid', 'GISUSER', 'GISUSER') #Database Connection
dbo2 = Dc.DBConnection('DOTDEV55SQL02', 'DataWarehouse', 'SHostetter', 'shostetter') #Database Connection



def get_intersection_universe(pg):
    cur = pg.conn.cursor()
    #.conn.cursor() Handles the connection to a PostgreSQL database instance. It encapsulates a database session. Return a new cursor object using the connection.

    cur.execute("select nodeid, masterid, is_int  from node where is_int = true")#Execute a database operation (query or command)
    # dictionary of nodeid: [masterid, intersection]
    nodes = defaultdict(list, ((int(row[0]), [row[1], row[2]]) for row in cur.fetchall()))
    # cur.fetchall() Fetch all (remaining) rows of a query result, returning them as a list of tuples. An empty list is returned if there is no more record to fetch.
    del cur
    #deletes cur to reset the memory for a new query
    return nodes

········

































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [2]:
node_dict = get_intersection_universe(pgo)

In [3]:
def alt_get_signalized_int(db, pg, nodes):
    cur = db.conn.cursor()
    cur.execute("""SELECT [PSGM_ID],[ContrType],[NodeID],[Longitude],[Latitude]
                    FROM [GISGRID].[gisadmin].[SIGNAL_CONTROLLER]
                    --[LION_CURRENT].[GISADMIN].[SIG_TRpsgm] 
                    where NormalizedType != 'Z'
                    --and [Longitude] not in ('','S' ) and  [Latitude] not in ('')
                    and [Longitude] is not null
                    --and psgm_id in (41084, 41083)""")
    for row in cur .fetchall():
        psgm_id, contrtype, nodeid, x, y = row
        nodeid = alt_get_node_from_signal_coords(pg, str(x), str(y))
        if nodeid:
            nodes[int(nodeid)].append('Signal')
    del cur
    return nodes


def alt_get_node_from_signal_coords(pg, x, y):
    cur = pg.conn.cursor()
    cur.execute("""select nodeid, st_distance(geom, st_transform(st_geomfromtext(
                        'SRID=4326;POINT(%s %s)'), 2263)) as distance
                    from node where st_distance(geom, st_transform(st_geomfromtext(
                        'SRID=4326;POINT(%s %s)'), 2263)) < 1000
                    and is_int = true
                    order by geom <#> st_transform(st_geomfromtext('SRID=4326;POINT(%s %s)'), 2263) limit 1
                """ % (x, y, x, y, x, y)
                )
    row = cur.fetchone()
    del cur
    if row:
        return int(row[0])
    else:
        return None
# make list of unsignalized masterids

In [ ]:
node_dict = alt_get_signalized_int(dbo, pgo, node_dict)

In [ ]:
def get_unsignalized_masterids(nodes):
    mids = defaultdict(list)
    to_remove = set()
    for node in nodes.keys():
        mids[nodes[node][0]].append(node)
        if nodes[node][-1] == 'Signal':
            to_remove.add(nodes[node][0])
    for m in to_remove:
        del(mids[m])
    return mids

In [ ]:
masters = get_unsignalized_masterids(node_dict)

In [ ]:
def get_all_crashes_nypd(db2):
    # test speed of doing 1 query and iterating in python memory
    cur = db2.conn.cursor()
    cur.execute("""SELECT distinct c.[ANUM_PCT]+c.[ANUM_YY]+c.[ANUM_SEQ] as crashid ,
                    c.[NODEID], cast(c.[OCCURRENCE_DATETIME] as DATE) as accd_date,
                    c.[INJURED_COUNT],c.[KILLED_COUNT],
                    case when vic.ped_nonped = 'P' then 1
                    when vic.PED_NONPED = 'B' then 2 else 3 end as inj_mode,
                    c.[ACCIDENT_DIAGRAM]  AS [COLLISION_TYP],
                    null as ext_of_inj, 0 as pre_action, 0 as direction, 0 as contrib
                    FROM [DataWarehouse].[dbo].[AIS_PD_Core_F] AS c
                    LEFT OUTER JOIN (
                        SELECT [ANUM_PCT], [ANUM_YY], [ANUM_SEQ],
                        MAX(CASE WHEN PRE_ACDNT_ACTION = '10' THEN 1
                        WHEN PRE_ACDNT_ACTION in ('15', '08', '09', '14', '04') THEN 1 -- removes other non-preventable
                        ELSE 0 END) AS PK
                        FROM [DataWarehouse].dbo.AIS_PD_Vehicle_F
                        GROUP BY [ANUM_PCT], [ANUM_YY], [ANUM_SEQ]
                    )AS v -- parked vehicles
                    ON c.[ANUM_PCT] = v.ANUM_PCT AND c.[ANUM_YY] = v.ANUM_YY AND c.[ANUM_SEQ] = v.ANUM_SEQ
                    LEFT OUTER JOIN DataWarehouse.dbo.AIS_PD_Pedestrian_F AS P
                    ON c.[ANUM_PCT] = P.ANUM_PCT AND c.[ANUM_YY] = P.ANUM_YY AND c.[ANUM_SEQ] = P.ANUM_SEQ
                    LEFT OUTER JOIN DataWarehouse.dbo.AIS_PD_Victim_F as vic
                    ON c.ANUM_PCT = vic.ANUM_PCT and c.ANUM_SEQ = vic.ANUM_SEQ and c.ANUM_YY = vic.ANUM_YY
                    WHERE C.OCCURRENCE_DATETIME > (select dateadd(week, -3, dateadd(year, -3, getdate())))
                    AND C.NODEID != 0
                    and v.PK != 1
                """)
    data = cur.fetchall()
    del cur
    # clean up dates
    for row in data:
        d = row[2]  # date field
        row[2] = datetime.datetime.strptime(d, "%Y-%m-%d").date()
    assert isinstance(data, object)
    return data

In [ ]:
crash_list = get_all_crashes_nypd(dbo2)  # NYPD data


In [11]:
def get_all_dir_right_angle_crashes_nypd(db2):
    cur = db2.conn.cursor()
    # cur.execute("""-- north / south
    #                 select distinct n.ANUM_PCT+ n.[ANUM_YY]+ n.[ANUM_SEQ] as crashid
    #                 --n.*, o.DIRECTION_OF_TRAVEL
    #                 from (
    #                     SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
    #                     FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
    #                     where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
    #                     and [DIRECTION_OF_TRAVEL] in (1, 5)
    #                     and PRE_ACDNT_ACTION not in ('09', '10', '14', '15')
    #                 ) as n
    #                 join (
    #                     SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
    #                     FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
    #                     where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
    #                     and [DIRECTION_OF_TRAVEL] not in (1, 5)
    #                     and PRE_ACDNT_ACTION not in ('09', '10', '14', '15')
    #                 ) as o
    #                 on n.ANUM_PCT = o.ANUM_PCT and n.ANUM_SEQ = o.ANUM_SEQ and n.ANUM_YY = o.ANUM_YY
    #                     and n.VEHICLE_NUMBER != o.VEHICLE_NUMBER
    #
    #                 union
    #                 -- east / west
    #                 select distinct n.ANUM_PCT+ n.[ANUM_YY]+ n.[ANUM_SEQ] as crashid
    #                 --n.*, o.DIRECTION_OF_TRAVEL
    #                 from (
    #                     SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
    #                     FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
    #                     where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
    #                     and [DIRECTION_OF_TRAVEL] in (3,7)
    #                     and PRE_ACDNT_ACTION not in ('09', '10', '14', '15')
    #                 ) as n
    #                 join (
    #                     SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
    #                     FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
    #                     where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
    #                     and [DIRECTION_OF_TRAVEL] not in (3,7)
    #                     and PRE_ACDNT_ACTION not in ('09', '10', '14', '15')
    #                 ) as o
    #                 on n.ANUM_PCT = o.ANUM_PCT and n.ANUM_SEQ = o.ANUM_SEQ and n.ANUM_YY = o.ANUM_YY
    #                     and n.VEHICLE_NUMBER != o.VEHICLE_NUMBER
    #
    #                 union
    #                 -- northeast / southwest
    #                 select distinct n.ANUM_PCT+ n.[ANUM_YY]+ n.[ANUM_SEQ] as crashid
    #                 --n.*, o.DIRECTION_OF_TRAVEL
    #                 from (
    #                     SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
    #                     FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
    #                     where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
    #                     and [DIRECTION_OF_TRAVEL] in (2, 6)
    #                     and PRE_ACDNT_ACTION not in ('09', '10', '14', '15')
    #                 ) as n
    #                 join (
    #                     SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
    #                     FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
    #                     where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
    #                     and [DIRECTION_OF_TRAVEL] not in (2, 6)
    #                     and PRE_ACDNT_ACTION not in ('09', '10', '14', '15')
    #                 ) as o
    #                 on n.ANUM_PCT = o.ANUM_PCT and n.ANUM_SEQ = o.ANUM_SEQ and n.ANUM_YY = o.ANUM_YY
    #                     and n.VEHICLE_NUMBER != o.VEHICLE_NUMBER
    #
    #                 union
    #                 -- northwest / southeast
    #                 select distinct n.ANUM_PCT+ n.[ANUM_YY]+ n.[ANUM_SEQ] as crashid
    #                 --n.*, o.DIRECTION_OF_TRAVEL
    #                 from (
    #                     SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
    #                     FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
    #                     where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
    #                     and [DIRECTION_OF_TRAVEL] in (4, 8)
    #                     and PRE_ACDNT_ACTION not in ('09', '10', '14', '15')
    #                 ) as n
    #                 join (
    #                     SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
    #                     FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
    #                     where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
    #                     and [DIRECTION_OF_TRAVEL] not in (4, 8)
    #                     and PRE_ACDNT_ACTION not in ('09', '10', '14', '15')
    #                 ) as o
    #                 on n.ANUM_PCT = o.ANUM_PCT and n.ANUM_SEQ = o.ANUM_SEQ and n.ANUM_YY = o.ANUM_YY
    #                     and n.VEHICLE_NUMBER != o.VEHICLE_NUMBER""")
    cur.execute("""/* conservative */
                    -- north / south
                    select distinct n.ANUM_PCT+ n.[ANUM_YY]+ n.[ANUM_SEQ] as crashid
                    --n.*, o.DIRECTION_OF_TRAVEL
                    from (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (1, 5)
                        and PRE_ACDNT_ACTION != '10'
                    ) as n
                    join (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (3, 7)
                    ) as o
                    on n.ANUM_PCT = o.ANUM_PCT and n.ANUM_SEQ = o.ANUM_SEQ and n.ANUM_YY = o.ANUM_YY
                        and n.VEHICLE_NUMBER != o.VEHICLE_NUMBER

                    union
                    -- east / west
                    select distinct n.ANUM_PCT+ n.[ANUM_YY]+ n.[ANUM_SEQ] as crashid
                    --n.*, o.DIRECTION_OF_TRAVEL
                    from (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (3,7)
                    ) as n
                    join (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (1, 5)
                    ) as o
                    on n.ANUM_PCT = o.ANUM_PCT and n.ANUM_SEQ = o.ANUM_SEQ and n.ANUM_YY = o.ANUM_YY
                        and n.VEHICLE_NUMBER != o.VEHICLE_NUMBER

                    union
                    -- northeast / southwest
                    select distinct n.ANUM_PCT+ n.[ANUM_YY]+ n.[ANUM_SEQ] as crashid
                    --n.*, o.DIRECTION_OF_TRAVEL
                    from (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (2, 6)
                    ) as n
                    join (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (4, 8)
                    ) as o
                    on n.ANUM_PCT = o.ANUM_PCT and n.ANUM_SEQ = o.ANUM_SEQ and n.ANUM_YY = o.ANUM_YY
                        and n.VEHICLE_NUMBER != o.VEHICLE_NUMBER

                    union
                    -- northwest / southeast
                    select distinct n.ANUM_PCT+ n.[ANUM_YY]+ n.[ANUM_SEQ] as crashid
                    --n.*, o.DIRECTION_OF_TRAVEL
                    from (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (4, 8)
                    ) as n
                    join (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (2, 6)
                    ) as o
                    on n.ANUM_PCT = o.ANUM_PCT and n.ANUM_SEQ = o.ANUM_SEQ and n.ANUM_YY = o.ANUM_YY
                        and n.VEHICLE_NUMBER != o.VEHICLE_NUMBER""")
    ra = cur.fetchall()
    del cur
    return set([i[0] for i in ra])


def get_all_dir_right_angle_crashes_nypd(db2):
    cur = db2.conn.cursor()
    cur.execute("""/* conservative */
                    -- north / south
                    select distinct n.ANUM_PCT+ n.[ANUM_YY]+ n.[ANUM_SEQ] as crashid
                    --n.*, o.DIRECTION_OF_TRAVEL
                    from (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (1, 5)
                        and PRE_ACDNT_ACTION != '10'
                    ) as n
                    join (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (3, 7)
                    ) as o
                    on n.ANUM_PCT = o.ANUM_PCT and n.ANUM_SEQ = o.ANUM_SEQ and n.ANUM_YY = o.ANUM_YY
                        and n.VEHICLE_NUMBER != o.VEHICLE_NUMBER

                    union
                    -- east / west
                    select distinct n.ANUM_PCT+ n.[ANUM_YY]+ n.[ANUM_SEQ] as crashid
                    --n.*, o.DIRECTION_OF_TRAVEL
                    from (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (3,7)
                    ) as n
                    join (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (1, 5)
                    ) as o
                    on n.ANUM_PCT = o.ANUM_PCT and n.ANUM_SEQ = o.ANUM_SEQ and n.ANUM_YY = o.ANUM_YY
                        and n.VEHICLE_NUMBER != o.VEHICLE_NUMBER

                    union
                    -- northeast / southwest
                    select distinct n.ANUM_PCT+ n.[ANUM_YY]+ n.[ANUM_SEQ] as crashid
                    --n.*, o.DIRECTION_OF_TRAVEL
                    from (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (2, 6)
                    ) as n
                    join (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (4, 8)
                    ) as o
                    on n.ANUM_PCT = o.ANUM_PCT and n.ANUM_SEQ = o.ANUM_SEQ and n.ANUM_YY = o.ANUM_YY
                        and n.VEHICLE_NUMBER != o.VEHICLE_NUMBER

                    union
                    -- northwest / southeast
                    select distinct n.ANUM_PCT+ n.[ANUM_YY]+ n.[ANUM_SEQ] as crashid
                    --n.*, o.DIRECTION_OF_TRAVEL
                    from (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (4, 8)
                    ) as n
                    join (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (2, 6)
                    ) as o
                    on n.ANUM_PCT = o.ANUM_PCT and n.ANUM_SEQ = o.ANUM_SEQ and n.ANUM_YY = o.ANUM_YY
                        and n.VEHICLE_NUMBER != o.VEHICLE_NUMBER""")
    ra = cur.fetchall()
    del cur
    return set([i[0] for i in ra])

In [12]:
drac = get_all_dir_right_angle_crashes_nypd(dbo2)  # NYPD data
drac

{u'0431702557',
 u'0431702556',
 u'1231610263',
 u'0521703263',
 u'0471703062',
 u'0661502913',
 u'0131701166',
 u'1211501863',
 u'0661502918',
 u'0471703069',
 u'0431702558',
 u'1051705931',
 u'0061700629',
 u'1111702065',
 u'0941702667',
 u'1041801232',
 u'1201600009',
 u'0671801232',
 u'0061800492',
 u'0061800493',
 u'1141703999',
 u'1091705848',
 u'0191610964',
 u'0421701104',
 u'0491610119',
 u'0181503926',
 u'0191610963',
 u'0751610061',
 u'1011600056',
 u'1081701185',
 u'0191503369',
 u'1141701848',
 u'1211800202',
 u'1041613421',
 u'1061600243',
 u'1061600241',
 u'0071610820',
 u'1211801156',
 u'1061612971',
 u'1211801152',
 u'0061800728',
 u'0141503683',
 u'0241501169',
 u'0431501973',
 u'0141503684',
 u'0191700191',
 u'0661800274',
 u'0661702131',
 u'0661702132',
 u'0941611311',
 u'0171801610',
 u'1111801828',
 u'1091704432',
 u'1201610900',
 u'1131501909',
 u'1091705235',
 u'1111502953',
 u'0701612115',
 u'0421500804',
 u'1081704980',
 u'1061703331',
 u'1001610284',
 u'07016

In [13]:
class Intersection:
    def __init__(self, masterid, nodeid_list, crashes_list):
        self.masterid = masterid
        self.nodeid_list = nodeid_list
        self.crashes = crashes_list
        self.one_year_crashes = dict()
        self.twelve_month_period()
        self.warrant()
        self.get_12_month_crashes()

    def five_or_more(self):
        count = 0
        for c in self.crashes:
            if c.preventable():
                count += 1
        if count > 4:
            return True
        else:
            return False

    def twelve_month_period(self):
        cl = [[c.crashid, c.ac_date] for c in self.crashes if c.preventable()]
        cl = sorted(cl, key=operator.itemgetter(1), reverse=True)
        while len(cl) > 1:
            if (cl[0][1] - cl[-1][1]).days / 365.0 > 1:
                cl.pop()
            else:
                break
        return cl

    def warrant(self):
        if len(self.twelve_month_period()) > 4:
            return True
        else:
            return False

    def get_12_month_crashes(self):
        crashes = dict((c[0], None) for c in self.twelve_month_period())
        for crash in self.crashes:
            if crash.crashid in crashes.keys():
                crashes[crash.crashid] = crash
        return crashes


In [14]:
def add_intersection_objects_to_master_dict(mids):
    for m in mids:
        mids[m].append(Intersection(m, mids[m], list()))
        if isinstance(mids[m][-1].nodeid_list[-1], Intersection):
            mids[m][-1].nodeid_list = mids[m][-1].nodeid_list[:-1]
            


In [15]:
add_intersection_objects_to_master_dict(masters)

In [16]:
class Crash:
    def __init__(self, crashid, nodeid, ac_date, inj, fat, mode, col_typ, sev, pre_actions, directions, contrib_fac):
        """
        :param crashid:
        :param nodeid:
        :param ac_date:
        :param inj:
        :param fat:
        :param mode:
        :param col_typ:
        :param sev:
        :param pre_actions:
        :param directions:
        :param contrib_fac:
        :return:
        """
        self.crashid = crashid
        self.nodeid = nodeid
        self.inj = inj
        self.fat = fat
        self.mode = mode
        self.col_typ = str(col_typ)
        self.pre_actions = pre_actions
        self.directions = directions
        self.ac_date = ac_date
        self.sev = sev
        self.contib_fac = contrib_fac
        self.right_angle()
        self.ped_bike()
        self.preventable()

    def right_angle(self):
        """
        includes: right angle (4), left turn with (0), and right turn with (5)
        :return: Boolean
        """
        if self.col_typ in ('4', '0', '5'):  # NYPD
            return True
        elif self.col_typ in ('04', '10', '05'):  # NYSDOT
            return True
        else:
            return False

    def ped_bike(self):
        if self.mode in (1, 2):
            return True
        else:
            return False

    def directions_prevent(self):
        if self.directions:
            return True
        else:
            return False

    def preventable(self):
        if self.right_angle() or self.ped_bike() or self.directions_prevent():
            if self.directions_prevent() and not self.right_angle():
                print '%s RA (%s), PB (%s), DIR(%s)' % (str(self.crashid),
                                                    str(self.right_angle()),
                                                    str(self.ped_bike()),
                                                    str(self.directions_prevent())
                                                    )
            return True
        else:
            return False

In [17]:
def add_crashes_to_intersections(nodes, mids, crashes, drac):
    for crash in crashes:
        crashid, nodeid, ac_date, inj, fat, mode, col_typ, sev, pre_actions, directions, contrib_fac = crash
        if crashid in drac:
            directions = True
        else:
            directions = False
        if nodes[int(nodeid)]:
            master = nodes[int(nodeid)][0]
            if master in mids.keys():  # check for unsignalized
                mids[master][-1].crashes.append(Crash(crashid, nodeid, ac_date, inj, fat, mode,
                                                      col_typ, sev, pre_actions, directions, contrib_fac))

In [18]:
add_crashes_to_intersections(node_dict, masters, crash_list, drac)


0011611627 RA (False), PB (False), DIR(True)
0011610559 RA (False), PB (False), DIR(True)
0011702551 RA (False), PB (False), DIR(True)
0011611331 RA (False), PB (True), DIR(True)
0011612151 RA (False), PB (False), DIR(True)
0011702882 RA (False), PB (False), DIR(True)
0011800840 RA (False), PB (False), DIR(True)
0011700617 RA (False), PB (False), DIR(True)
0051610689 RA (False), PB (False), DIR(True)
0051612101 RA (False), PB (False), DIR(True)
0051701769 RA (False), PB (False), DIR(True)
0011801344 RA (False), PB (False), DIR(True)
0061610961 RA (False), PB (True), DIR(True)
0061611215 RA (False), PB (False), DIR(True)
0061700641 RA (False), PB (False), DIR(True)
0071700672 RA (False), PB (False), DIR(True)
0071700870 RA (False), PB (False), DIR(True)
0071701089 RA (False), PB (False), DIR(True)
0071701215 RA (False), PB (False), DIR(True)
0071701332 RA (False), PB (True), DIR(True)
0071800698 RA (False), PB (False), DIR(True)
0011611198 RA (False), PB (False), DIR(True)
0061611016 RA

0341800869 RA (False), PB (False), DIR(True)
0431802385 RA (False), PB (False), DIR(True)
0441610880 RA (False), PB (False), DIR(True)
0441611372 RA (False), PB (False), DIR(True)
0401611239 RA (False), PB (False), DIR(True)
0401611282 RA (False), PB (False), DIR(True)
0401611369 RA (False), PB (False), DIR(True)
0401611574 RA (False), PB (False), DIR(True)
0401611792 RA (False), PB (False), DIR(True)
0401702256 RA (False), PB (False), DIR(True)
0441700002 RA (False), PB (False), DIR(True)
0401702878 RA (False), PB (False), DIR(True)
0441700637 RA (False), PB (False), DIR(True)
0401703581 RA (False), PB (False), DIR(True)
0401612002 RA (False), PB (False), DIR(True)
0401612045 RA (False), PB (False), DIR(True)
0441701492 RA (False), PB (True), DIR(True)
0441702001 RA (False), PB (False), DIR(True)
0441702094 RA (False), PB (False), DIR(True)
0441702231 RA (False), PB (False), DIR(True)
0441702515 RA (False), PB (False), DIR(True)
0441703083 RA (False), PB (False), DIR(True)
0441703135 

0441800200 RA (False), PB (False), DIR(True)
0471701282 RA (False), PB (False), DIR(True)
0471701342 RA (False), PB (False), DIR(True)
0471701672 RA (False), PB (False), DIR(True)
0471701736 RA (False), PB (False), DIR(True)
0471701860 RA (False), PB (False), DIR(True)
0471701925 RA (False), PB (False), DIR(True)
0431610614 RA (False), PB (False), DIR(True)
0441800531 RA (False), PB (False), DIR(True)
0441800599 RA (False), PB (False), DIR(True)
0471702124 RA (False), PB (False), DIR(True)
0471702143 RA (False), PB (False), DIR(True)
0471702371 RA (False), PB (False), DIR(True)
0471702587 RA (False), PB (False), DIR(True)
0471702589 RA (False), PB (False), DIR(True)
0471702600 RA (False), PB (False), DIR(True)
0441801726 RA (False), PB (False), DIR(True)
0431611456 RA (False), PB (False), DIR(True)
0431611917 RA (False), PB (False), DIR(True)
0431611954 RA (False), PB (False), DIR(True)
0451611077 RA (False), PB (False), DIR(True)
0451611933 RA (False), PB (False), DIR(True)
0451612037

0471702905 RA (False), PB (False), DIR(True)
0471702957 RA (False), PB (False), DIR(True)
0471703260 RA (False), PB (False), DIR(True)
0471703324 RA (False), PB (False), DIR(True)
0471703337 RA (False), PB (False), DIR(True)
0471703446 RA (False), PB (False), DIR(True)
0471703632 RA (False), PB (False), DIR(True)
0471703705 RA (False), PB (False), DIR(True)
0471703809 RA (False), PB (False), DIR(True)
0471704023 RA (False), PB (False), DIR(True)
0471704118 RA (False), PB (False), DIR(True)
0471800012 RA (False), PB (False), DIR(True)
0471800105 RA (False), PB (False), DIR(True)
0471800224 RA (False), PB (False), DIR(True)
0471800420 RA (False), PB (False), DIR(True)
0471800898 RA (False), PB (True), DIR(True)
0471801084 RA (False), PB (False), DIR(True)
0471801557 RA (False), PB (False), DIR(True)
0471801728 RA (False), PB (False), DIR(True)
0471801745 RA (False), PB (False), DIR(True)
0471802229 RA (False), PB (False), DIR(True)
0471802281 RA (False), PB (False), DIR(True)
0481610382 

0481701198 RA (False), PB (False), DIR(True)
0481701696 RA (False), PB (False), DIR(True)
0501800435 RA (False), PB (False), DIR(True)
0501800440 RA (False), PB (False), DIR(True)
0501800445 RA (False), PB (False), DIR(True)
0481702155 RA (False), PB (False), DIR(True)
0481702276 RA (False), PB (True), DIR(True)
0501801192 RA (False), PB (False), DIR(True)
0501801281 RA (False), PB (False), DIR(True)
0501801380 RA (False), PB (False), DIR(True)
0481702554 RA (False), PB (False), DIR(True)
0481702703 RA (False), PB (False), DIR(True)
0481800382 RA (False), PB (False), DIR(True)
0481800679 RA (False), PB (False), DIR(True)
0521701930 RA (False), PB (False), DIR(True)
0521611661 RA (False), PB (False), DIR(True)
0521611677 RA (False), PB (False), DIR(True)
0481800974 RA (False), PB (False), DIR(True)
0481801012 RA (False), PB (False), DIR(True)
0481801136 RA (False), PB (False), DIR(True)
0481801269 RA (False), PB (False), DIR(True)
0521702363 RA (False), PB (False), DIR(True)
0521702748 

...

0671701333 RA (False), PB (False), DIR(True)
0671701563 RA (False), PB (False), DIR(True)
0671701706 RA (False), PB (False), DIR(True)
0671701953 RA (False), PB (False), DIR(True)
0671702165 RA (False), PB (False), DIR(True)
0671702215 RA (False), PB (False), DIR(True)
0671703076 RA (False), PB (False), DIR(True)
0671703176 RA (False), PB (False), DIR(True)
0671703980 RA (False), PB (False), DIR(True)
0671703999 RA (False), PB (False), DIR(True)
0671800149 RA (False), PB (False), DIR(True)
0671801234 RA (False), PB (False), DIR(True)
0671801828 RA (False), PB (False), DIR(True)
0681610524 RA (False), PB (False), DIR(True)
0681610932 RA (False), PB (False), DIR(True)
0681611004 RA (False), PB (False), DIR(True)
0681611125 RA (False), PB (False), DIR(True)
0681611668 RA (False), PB (False), DIR(True)
0681611903 RA (False), PB (False), DIR(True)
0681612256 RA (False), PB (False), DIR(True)
0681700459 RA (False), PB (False), DIR(True)
0681700991 RA (False), PB (False), DIR(True)
0681701040

0761701472 RA (False), PB (False), DIR(True)
0761800513 RA (False), PB (False), DIR(True)
0771800360 RA (False), PB (False), DIR(True)
0771800965 RA (False), PB (False), DIR(True)
0781800983 RA (False), PB (False), DIR(True)
0791700013 RA (False), PB (False), DIR(True)
0791800973 RA (False), PB (False), DIR(True)
0811800556 RA (False), PB (False), DIR(True)
0811800678 RA (False), PB (False), DIR(True)
0831610628 RA (False), PB (False), DIR(True)
0831611204 RA (False), PB (False), DIR(True)
0831611511 RA (False), PB (False), DIR(True)
0831700531 RA (False), PB (False), DIR(True)
0831700549 RA (False), PB (False), DIR(True)
0831700700 RA (False), PB (False), DIR(True)
0831700938 RA (False), PB (False), DIR(True)
0831700978 RA (False), PB (False), DIR(True)
0831701237 RA (False), PB (True), DIR(True)
0831701315 RA (False), PB (True), DIR(True)
0831701855 RA (False), PB (False), DIR(True)
0831702441 RA (False), PB (False), DIR(True)
0831800037 RA (False), PB (False), DIR(True)
0831800525 R

1041704273 RA (False), PB (False), DIR(True)
1041704294 RA (False), PB (False), DIR(True)
1041800108 RA (False), PB (False), DIR(True)
1041800568 RA (False), PB (False), DIR(True)
1041800696 RA (False), PB (False), DIR(True)
1041801027 RA (False), PB (False), DIR(True)
1041801222 RA (False), PB (False), DIR(True)
1041801656 RA (False), PB (False), DIR(True)
1041801679 RA (False), PB (False), DIR(True)
1041802044 RA (False), PB (False), DIR(True)
1041802115 RA (False), PB (False), DIR(True)
1041802297 RA (False), PB (False), DIR(True)
1041802409 RA (False), PB (False), DIR(True)
1041802536 RA (False), PB (False), DIR(True)
1051611014 RA (False), PB (False), DIR(True)
1051611074 RA (False), PB (False), DIR(True)
1051611149 RA (False), PB (False), DIR(True)
1051611438 RA (False), PB (False), DIR(True)
1051611712 RA (False), PB (False), DIR(True)
1051611913 RA (False), PB (False), DIR(True)
1051612878 RA (False), PB (False), DIR(True)
1051612879 RA (False), PB (False), DIR(True)
1051612888

1091614781 RA (False), PB (False), DIR(True)
1091614820 RA (False), PB (False), DIR(True)
1091614846 RA (False), PB (False), DIR(True)
1091615284 RA (False), PB (False), DIR(True)
1091615794 RA (False), PB (False), DIR(True)
1091615832 RA (False), PB (False), DIR(True)
1091700224 RA (False), PB (False), DIR(True)
1091700322 RA (False), PB (False), DIR(True)
1091700600 RA (False), PB (False), DIR(True)
1091700688 RA (False), PB (True), DIR(True)
1091700869 RA (False), PB (False), DIR(True)
1091700889 RA (False), PB (False), DIR(True)
1091700904 RA (False), PB (False), DIR(True)
1091701118 RA (False), PB (False), DIR(True)
1091701353 RA (False), PB (False), DIR(True)
1091701355 RA (False), PB (False), DIR(True)
1091701429 RA (False), PB (False), DIR(True)
1091701770 RA (False), PB (False), DIR(True)
1091701874 RA (False), PB (False), DIR(True)
1091702003 RA (False), PB (False), DIR(True)
1091702262 RA (False), PB (False), DIR(True)
1091702280 RA (False), PB (False), DIR(True)
1091702425 

1131801204 RA (False), PB (False), DIR(True)
1131801401 RA (False), PB (False), DIR(True)
1131801481 RA (False), PB (False), DIR(True)
1131801500 RA (False), PB (False), DIR(True)
1131802145 RA (False), PB (False), DIR(True)
1131802216 RA (False), PB (False), DIR(True)
1131802308 RA (False), PB (False), DIR(True)
1131802427 RA (False), PB (False), DIR(True)
1141611185 RA (False), PB (False), DIR(True)
1141611566 RA (False), PB (False), DIR(True)
1141612278 RA (False), PB (False), DIR(True)
1141612363 RA (False), PB (False), DIR(True)
1141612813 RA (False), PB (False), DIR(True)
1141612836 RA (False), PB (False), DIR(True)
1141612912 RA (False), PB (False), DIR(True)
1141613296 RA (False), PB (False), DIR(True)
1141613302 RA (False), PB (False), DIR(True)
1141613354 RA (False), PB (False), DIR(True)
1141613543 RA (False), PB (False), DIR(True)
1141613810 RA (False), PB (False), DIR(True)
1141614080 RA (False), PB (False), DIR(True)
1141700364 RA (False), PB (False), DIR(True)
1141700699

In [19]:
def approved_intersections(md):
    ints_w_warrant_met = dict()
    for master in md.keys():
        if md[master][-1].warrant():
            ints_w_warrant_met[master] = md[master][-1].get_12_month_crashes()
            # print master, len(md[master][-1].twelve_month_period())
    return ints_w_warrant_met

In [20]:
approved_masters = approved_intersections(masters)

1231611191 RA (False), PB (True), DIR(True)
1231610296 RA (False), PB (False), DIR(True)
1231700896 RA (False), PB (False), DIR(True)
1231800572 RA (False), PB (False), DIR(True)
1231800702 RA (False), PB (False), DIR(True)
1231610599 RA (False), PB (False), DIR(True)
1231800725 RA (False), PB (False), DIR(True)
1231610915 RA (False), PB (False), DIR(True)
1231801116 RA (False), PB (False), DIR(True)
1231800389 RA (False), PB (False), DIR(True)
1231800143 RA (False), PB (False), DIR(True)
1231800491 RA (False), PB (False), DIR(True)
1231701028 RA (False), PB (False), DIR(True)
1231700783 RA (False), PB (False), DIR(True)
1231701475 RA (False), PB (False), DIR(True)
1231610830 RA (False), PB (False), DIR(True)
1231800023 RA (False), PB (False), DIR(True)
1231701123 RA (False), PB (False), DIR(True)
1231801000 RA (False), PB (False), DIR(True)
1231611059 RA (False), PB (False), DIR(True)
1231611522 RA (False), PB (False), DIR(True)
1231800111 RA (False), PB (False), DIR(True)
1231800892 

0721702508 RA (False), PB (False), DIR(True)
0721702899 RA (False), PB (False), DIR(True)
0721702508 RA (False), PB (False), DIR(True)
0721702899 RA (False), PB (False), DIR(True)
0661800864 RA (False), PB (True), DIR(True)
0661800432 RA (False), PB (False), DIR(True)
0661800259 RA (False), PB (False), DIR(True)
0721700639 RA (False), PB (False), DIR(True)
0721701526 RA (False), PB (False), DIR(True)
0721700639 RA (False), PB (False), DIR(True)
0721701526 RA (False), PB (False), DIR(True)
0721610832 RA (False), PB (False), DIR(True)
0721703013 RA (False), PB (False), DIR(True)
0721800512 RA (False), PB (False), DIR(True)
0721611246 RA (False), PB (True), DIR(True)
0721701063 RA (False), PB (False), DIR(True)
0721701162 RA (False), PB (False), DIR(True)
0661611093 RA (False), PB (False), DIR(True)
0661702224 RA (False), PB (True), DIR(True)
0761610875 RA (False), PB (False), DIR(True)
0761701397 RA (False), PB (False), DIR(True)
0761611171 RA (False), PB (False), DIR(True)
0761700093 RA

0631801283 RA (False), PB (False), DIR(True)
0631801654 RA (False), PB (False), DIR(True)
0631611095 RA (False), PB (False), DIR(True)
0631611022 RA (False), PB (True), DIR(True)
0631611329 RA (False), PB (False), DIR(True)
0631611329 RA (False), PB (False), DIR(True)
0631610719 RA (False), PB (False), DIR(True)
0631700495 RA (False), PB (False), DIR(True)
0631801014 RA (False), PB (False), DIR(True)
0631700715 RA (False), PB (False), DIR(True)
0631610413 RA (False), PB (False), DIR(True)
0631701197 RA (False), PB (False), DIR(True)
0631801346 RA (False), PB (False), DIR(True)
0631801625 RA (False), PB (False), DIR(True)
0631700777 RA (False), PB (False), DIR(True)
0691611226 RA (False), PB (False), DIR(True)
0691800349 RA (False), PB (False), DIR(True)
0671801901 RA (False), PB (False), DIR(True)
0671701953 RA (False), PB (False), DIR(True)
0671703434 RA (False), PB (False), DIR(True)
0671700704 RA (False), PB (False), DIR(True)
0671611822 RA (False), PB (False), DIR(True)
0671701210 

0751701320 RA (False), PB (False), DIR(True)
0751701320 RA (False), PB (False), DIR(True)
0751702001 RA (False), PB (False), DIR(True)
0751705228 RA (False), PB (False), DIR(True)
0751705228 RA (False), PB (False), DIR(True)
0751703070 RA (False), PB (False), DIR(True)
0751703464 RA (False), PB (False), DIR(True)
0751800315 RA (False), PB (False), DIR(True)
0751703930 RA (False), PB (True), DIR(True)
0751705579 RA (False), PB (False), DIR(True)
0751802530 RA (False), PB (True), DIR(True)
0751703930 RA (False), PB (True), DIR(True)
0751705579 RA (False), PB (False), DIR(True)
0751802530 RA (False), PB (True), DIR(True)
0751705708 RA (False), PB (False), DIR(True)
0751701887 RA (False), PB (False), DIR(True)
0831800027 RA (False), PB (False), DIR(True)
1041701663 RA (False), PB (False), DIR(True)
1041800001 RA (False), PB (False), DIR(True)
1041610973 RA (False), PB (True), DIR(True)
1041612967 RA (False), PB (False), DIR(True)
1041801027 RA (False), PB (False), DIR(True)
1041612967 RA (

1131701775 RA (False), PB (False), DIR(True)
1131703670 RA (False), PB (False), DIR(True)
1131611622 RA (False), PB (False), DIR(True)
1131704080 RA (False), PB (False), DIR(True)
1031701707 RA (False), PB (False), DIR(True)
1031700836 RA (False), PB (False), DIR(True)
1071704335 RA (False), PB (True), DIR(True)
1131701205 RA (False), PB (False), DIR(True)
1131702343 RA (False), PB (True), DIR(True)
1131702343 RA (False), PB (True), DIR(True)
1031612230 RA (False), PB (False), DIR(True)
1131802308 RA (False), PB (False), DIR(True)
1131612825 RA (False), PB (False), DIR(True)
1031611184 RA (False), PB (False), DIR(True)
1031611588 RA (False), PB (False), DIR(True)
1031611184 RA (False), PB (False), DIR(True)
1031611588 RA (False), PB (False), DIR(True)
1031612722 RA (False), PB (False), DIR(True)
1031702704 RA (False), PB (False), DIR(True)
1031700860 RA (False), PB (True), DIR(True)
1031611919 RA (False), PB (False), DIR(True)
1031801611 RA (False), PB (False), DIR(True)
1031701598 RA 

0401612499 RA (False), PB (False), DIR(True)
0401800153 RA (False), PB (False), DIR(True)
0421701339 RA (False), PB (False), DIR(True)
0421700605 RA (False), PB (False), DIR(True)
0301800342 RA (False), PB (False), DIR(True)
0331610731 RA (False), PB (False), DIR(True)
0331700527 RA (False), PB (False), DIR(True)
0331700372 RA (False), PB (False), DIR(True)
0341701485 RA (False), PB (False), DIR(True)
0321700043 RA (False), PB (False), DIR(True)
0331610441 RA (False), PB (False), DIR(True)
0341700081 RA (False), PB (False), DIR(True)
0341701185 RA (False), PB (True), DIR(True)
0341701745 RA (False), PB (False), DIR(True)
0441611236 RA (False), PB (False), DIR(True)
0441702146 RA (False), PB (False), DIR(True)
0441701139 RA (False), PB (True), DIR(True)
0441800313 RA (False), PB (False), DIR(True)
0441801543 RA (False), PB (False), DIR(True)
0441800200 RA (False), PB (False), DIR(True)
0441800400 RA (False), PB (False), DIR(True)
0441612061 RA (False), PB (False), DIR(True)
0441800430 R

1101611761 RA (False), PB (True), DIR(True)
1121700028 RA (False), PB (False), DIR(True)
1121700571 RA (False), PB (False), DIR(True)
1121700756 RA (False), PB (False), DIR(True)
1101701982 RA (False), PB (False), DIR(True)
1121801425 RA (False), PB (False), DIR(True)
1121612279 RA (False), PB (False), DIR(True)
1121702551 RA (False), PB (False), DIR(True)
1121611701 RA (False), PB (False), DIR(True)
1121800440 RA (False), PB (False), DIR(True)
1121702825 RA (False), PB (False), DIR(True)
1121701662 RA (False), PB (False), DIR(True)
1121610603 RA (False), PB (False), DIR(True)
1121702070 RA (False), PB (False), DIR(True)
1121701213 RA (False), PB (False), DIR(True)
1121701434 RA (False), PB (False), DIR(True)
1121800535 RA (False), PB (False), DIR(True)
1121800730 RA (False), PB (False), DIR(True)
1121800535 RA (False), PB (False), DIR(True)
1121800730 RA (False), PB (False), DIR(True)
1121611246 RA (False), PB (False), DIR(True)
1121611116 RA (False), PB (False), DIR(True)
1121703026 

1091707580 RA (False), PB (False), DIR(True)
1091802019 RA (False), PB (False), DIR(True)
1091613288 RA (False), PB (True), DIR(True)
1091800938 RA (False), PB (False), DIR(True)
1091801481 RA (False), PB (False), DIR(True)
1091706697 RA (False), PB (False), DIR(True)
1091700505 RA (False), PB (False), DIR(True)
1091702788 RA (False), PB (False), DIR(True)
1091613084 RA (False), PB (True), DIR(True)
1091703358 RA (False), PB (False), DIR(True)
1091803045 RA (False), PB (False), DIR(True)
1091705711 RA (False), PB (False), DIR(True)
1091613084 RA (False), PB (True), DIR(True)
1091703358 RA (False), PB (False), DIR(True)
1091803045 RA (False), PB (False), DIR(True)
1091705711 RA (False), PB (False), DIR(True)
1091613230 RA (False), PB (False), DIR(True)
1091613168 RA (False), PB (True), DIR(True)
1091803534 RA (False), PB (False), DIR(True)
1091613648 RA (False), PB (False), DIR(True)
1091612422 RA (False), PB (False), DIR(True)
1091705214 RA (False), PB (False), DIR(True)
1091611128 RA 

In [21]:
def get_node_details(pg):
    data = dict()
    cur = pg.conn.cursor()
    cur.execute("""select  nodeid, masterid, st1, st2, st_x(geom) as x, st_y(geom) as y
                    from node n
                    join (
                        select nodeidfrom, min(st1) as st1, max(st2) as st2
                        from(
                            select s1.nodeidfrom, s1.street as st1, s2.street  as st2
                            from lion as s1 join lion as s2
                            on s1.nodeidfrom = s2.nodeidto
                            where s1.street != s2.street
                            and s1.featuretyp not in ('1', '2', '3', '7')
                            and s2.featuretyp not in ('1', '2', '3', '7')
                        ) as names_ group by nodeidfrom
                    ) as st_names on n.nodeid::int = st_names.nodeidfrom::int
                """)
    for row in cur.fetchall():
        nodeid, masterid, st1, st2, x, y = row
        data[nodeid] = [st1, st2, x, y]
    del cur
    return data

In [22]:
node_data = get_node_details(pgo)
node_data

{Decimal('9043971'): ['CEDAR AVENUE',
  'SEDGWICK AVENUE',
  1006251.0139955,
  249834.141871974],
 Decimal('9043976'): ['DRIVEWAY',
  'CLIFTON AVENUE',
  963095.125501141,
  161331.486815557],
 Decimal('10'): ['WARDS POINT AVENUE',
  'HYLAN BOULEVARD',
  913671.212089136,
  122413.678241149],
 Decimal('11'): ['CANTER AVENUE',
  'HYLAN BOULEVARD',
  913966.704595372,
  122458.842518911],
 Decimal('14'): ['WARDS POINT AVENUE',
  'PERTH AMBOY PLACE',
  913553.959376037,
  123788.918540359],
 Decimal('9043983'): ['ALLEY', 'NEW LANE', 966923.552715108, 162602.589860439],
 Decimal('18'): ['SATTERLEE STREET',
  'SHORE ROAD',
  913844.414819092,
  123550.700851426],
 Decimal('19'): ['PERTH AMBOY PLACE',
  'SATTERLEE STREET',
  913868.315688878,
  123836.428285763],
 Decimal('22'): ['WARDS POINT AVENUE',
  'TOTTENVILLE PLACE',
  913469.290913939,
  124077.418606833],
 Decimal('23'): ['AVIVA COURT',
  'AMBOY ROAD',
  913353.156964973,
  124359.711008817],
 Decimal('25'): ['AMBOY ROAD',
  'HOPPI

In [23]:
for i in node_data:
    if i == 33755:
     print node_data[i]

['ROCKAWAY BOULEVARD', '110 STREET', 1031726.09807865, 186224.401813105]


In [24]:
def intersection_summaries(approved, mids, nds):
    summary = list()
    for m in approved.keys():
        crashes = len([c for c in approved[m]])
        ped_inj = sum([approved[m][c].inj for c in approved[m] if approved[m][c].mode == 1])
        bike_inj = sum([approved[m][c].inj for c in approved[m] if approved[m][c].mode == 2])
        mvo_inj = sum([approved[m][c].inj for c in approved[m] if approved[m][c].mode == 3])
        right_angles = len([approved[m][c].crashid for c in approved[m] if approved[m][c].col_typ in ('4', '04')])
        date_ranges = [min([approved[m][c].ac_date for c in approved[m]]),
                       max([approved[m][c].ac_date for c in approved[m]])]
        print 'Master %i (%s - %s):\n\tNodes %s \n\t%i crashes\n\t%i ped injuries\n\t%i bike' \
              'injuries\n\t%i mvo injuries\n\t%i right angle crashes' % (m, date_ranges[0].strftime('%m/%d/%Y'),
                                                                         date_ranges[1].strftime('%m/%d/%Y'),
                                                                         str(mids[m][-1].nodeid_list),
                                                                         crashes, ped_inj, bike_inj, mvo_inj,
                                                                         right_angles)
        if len(mids[m]) > 2:
            for node in mids[m][:-1]:
                if node in nds.keys():
                    s1, s2, x, y = nds[node]
                else:
                    s1, s2, x, y = '', '', 0, 0
                summary.append([m, node, crashes, ped_inj, bike_inj, mvo_inj, ped_inj+bike_inj+mvo_inj,
                                right_angles, date_ranges[0], date_ranges[1], s1, s2, x, y])
        else:
            if mids[m][0] in nds.keys():
                s1, s2, x, y = nds[mids[m][0]]
            else:
                s1, s2, x, y = '', '', 0, 0
            summary.append([m, mids[m][0], crashes, ped_inj, bike_inj, mvo_inj, ped_inj+bike_inj+mvo_inj,
                            right_angles, date_ranges[0], date_ranges[1], s1, s2, x, y])
    return ['MasterID', 'NodeID', 'Crashes', 'Ped_Injs', 'Bike_Injs', 'MVO_Injs', 'Total_Injs',
            'Right_Angle_Crashes', 'Earliest_Date', 'Latest_Date', 'Street1', 'Street2', 'X', 'Y'], summary

In [27]:
headers, crash_data = intersection_summaries(approved_masters, masters, node_data)


Master 26626 (07/24/2017 - 05/06/2018):
	Nodes [26626] 
	6 crashes
	1 ped injuries
	0 bikeinjuries
	4 mvo injuries
	2 right angle crashes
Master 18439 (06/10/2017 - 06/08/2018):
	Nodes [18439] 
	5 crashes
	0 ped injuries
	1 bikeinjuries
	0 mvo injuries
	2 right angle crashes
Master 18440 (03/02/2017 - 01/25/2018):
	Nodes [18440] 
	5 crashes
	0 ped injuries
	0 bikeinjuries
	1 mvo injuries
	3 right angle crashes
Master 32786 (08/18/2017 - 05/16/2018):
	Nodes [32786] 
	6 crashes
	1 ped injuries
	0 bikeinjuries
	2 mvo injuries
	5 right angle crashes
Master 20500 (06/22/2016 - 01/11/2017):
	Nodes [20500] 
	5 crashes
	2 ped injuries
	0 bikeinjuries
	0 mvo injuries
	1 right angle crashes
Master 36885 (08/10/2017 - 07/12/2018):
	Nodes [36885] 
	5 crashes
	1 ped injuries
	0 bikeinjuries
	3 mvo injuries
	3 right angle crashes
Master 36888 (08/02/2017 - 07/15/2018):
	Nodes [36888] 
	5 crashes
	0 ped injuries
	0 bikeinjuries
	0 mvo injuries
	4 right angle crashes
Master 55321 (11/27/2017 - 08/04/2

Master 24827 (06/23/2017 - 03/11/2018):
	Nodes [24827] 
	6 crashes
	0 ped injuries
	0 bikeinjuries
	0 mvo injuries
	5 right angle crashes
Master 51454 (08/01/2017 - 07/13/2018):
	Nodes [51454] 
	11 crashes
	2 ped injuries
	2 bikeinjuries
	1 mvo injuries
	2 right angle crashes
Master 24839 (01/15/2017 - 08/25/2017):
	Nodes [24839] 
	5 crashes
	0 ped injuries
	1 bikeinjuries
	0 mvo injuries
	3 right angle crashes
Master 65802 (06/23/2017 - 04/09/2018):
	Nodes [27798, 27799, 27800, 65802] 
	5 crashes
	2 ped injuries
	0 bikeinjuries
	0 mvo injuries
	2 right angle crashes
Master 67318 (07/30/2017 - 06/23/2018):
	Nodes [34046, 67318] 
	5 crashes
	0 ped injuries
	0 bikeinjuries
	2 mvo injuries
	3 right angle crashes
Master 10516 (08/25/2016 - 02/21/2017):
	Nodes [10514, 10516] 
	5 crashes
	1 ped injuries
	0 bikeinjuries
	1 mvo injuries
	5 right angle crashes
Master 49436 (06/29/2017 - 06/04/2018):
	Nodes [49436] 
	6 crashes
	0 ped injuries
	1 bikeinjuries
	0 mvo injuries
	4 right angle crashe

Master 31211 (07/11/2017 - 06/22/2018):
	Nodes [31211] 
	5 crashes
	0 ped injuries
	2 bikeinjuries
	1 mvo injuries
	3 right angle crashes
Master 39406 (12/16/2016 - 07/26/2017):
	Nodes [39406] 
	5 crashes
	0 ped injuries
	0 bikeinjuries
	1 mvo injuries
	3 right angle crashes
Master 31215 (01/23/2017 - 07/25/2017):
	Nodes [31215] 
	5 crashes
	0 ped injuries
	0 bikeinjuries
	2 mvo injuries
	3 right angle crashes
Master 27123 (04/16/2018 - 06/09/2018):
	Nodes [27123] 
	6 crashes
	3 ped injuries
	0 bikeinjuries
	0 mvo injuries
	2 right angle crashes
Master 41464 (06/02/2016 - 04/25/2017):
	Nodes [41464] 
	6 crashes
	1 ped injuries
	0 bikeinjuries
	0 mvo injuries
	3 right angle crashes
Master 23035 (08/26/2016 - 08/25/2017):
	Nodes [23035] 
	10 crashes
	0 ped injuries
	0 bikeinjuries
	7 mvo injuries
	6 right angle crashes
Master 37372 (06/01/2017 - 05/08/2018):
	Nodes [37372] 
	5 crashes
	0 ped injuries
	0 bikeinjuries
	5 mvo injuries
	4 right angle crashes
Master 41470 (10/09/2017 - 06/24/

Master 49836 (08/24/2017 - 07/09/2018):
	Nodes [49836, 81781, 87799] 
	7 crashes
	3 ped injuries
	0 bikeinjuries
	2 mvo injuries
	1 right angle crashes
Master 12974 (06/22/2017 - 03/27/2018):
	Nodes [12974] 
	6 crashes
	0 ped injuries
	0 bikeinjuries
	4 mvo injuries
	5 right angle crashes
Master 27317 (08/29/2017 - 05/29/2018):
	Nodes [27317] 
	6 crashes
	0 ped injuries
	1 bikeinjuries
	4 mvo injuries
	5 right angle crashes
Master 27320 (02/04/2017 - 11/02/2017):
	Nodes [27320] 
	6 crashes
	0 ped injuries
	0 bikeinjuries
	0 mvo injuries
	3 right angle crashes
Master 44148 (09/11/2017 - 05/04/2018):
	Nodes [44148] 
	5 crashes
	2 ped injuries
	0 bikeinjuries
	0 mvo injuries
	0 right angle crashes
Master 53706 (08/01/2017 - 04/17/2018):
	Nodes [53706] 
	7 crashes
	1 ped injuries
	2 bikeinjuries
	4 mvo injuries
	3 right angle crashes
Master 47809 (12/18/2016 - 09/17/2017):
	Nodes [47809] 
	8 crashes
	0 ped injuries
	0 bikeinjuries
	7 mvo injuries
	3 right angle crashes
Master 29380 (09/23/

Master 52140 (07/02/2017 - 06/13/2018):
	Nodes [52140] 
	6 crashes
	0 ped injuries
	0 bikeinjuries
	4 mvo injuries
	6 right angle crashes
Master 29613 (08/08/2017 - 07/14/2018):
	Nodes [29613] 
	5 crashes
	1 ped injuries
	0 bikeinjuries
	1 mvo injuries
	3 right angle crashes
Master 45998 (05/24/2017 - 05/13/2018):
	Nodes [45998] 
	5 crashes
	0 ped injuries
	0 bikeinjuries
	3 mvo injuries
	2 right angle crashes
Master 54195 (09/09/2017 - 06/12/2018):
	Nodes [54195] 
	8 crashes
	0 ped injuries
	0 bikeinjuries
	6 mvo injuries
	3 right angle crashes
Master 43956 (09/06/2017 - 07/07/2018):
	Nodes [43956] 
	8 crashes
	2 ped injuries
	0 bikeinjuries
	0 mvo injuries
	4 right angle crashes
Master 48053 (06/01/2017 - 05/22/2018):
	Nodes [48053] 
	5 crashes
	0 ped injuries
	0 bikeinjuries
	0 mvo injuries
	2 right angle crashes
Master 51017 (12/12/2016 - 11/18/2017):
	Nodes [51017] 
	5 crashes
	0 ped injuries
	0 bikeinjuries
	5 mvo injuries
	2 right angle crashes
Master 48655 (09/15/2017 - 07/21/2

Master 50236 (09/07/2017 - 06/09/2018):
	Nodes [50236, 86223, 86302] 
	8 crashes
	0 ped injuries
	0 bikeinjuries
	2 mvo injuries
	5 right angle crashes
Master 54333 (05/05/2017 - 03/25/2018):
	Nodes [54333] 
	6 crashes
	0 ped injuries
	0 bikeinjuries
	3 mvo injuries
	5 right angle crashes
Master 54339 (03/10/2017 - 01/24/2018):
	Nodes [54339] 
	5 crashes
	2 ped injuries
	1 bikeinjuries
	0 mvo injuries
	1 right angle crashes
Master 13153 (08/18/2017 - 05/27/2018):
	Nodes [13153] 
	5 crashes
	0 ped injuries
	0 bikeinjuries
	3 mvo injuries
	3 right angle crashes
Master 19530 (04/20/2017 - 04/11/2018):
	Nodes [19530] 
	6 crashes
	0 ped injuries
	0 bikeinjuries
	5 mvo injuries
	4 right angle crashes
Master 29782 (11/18/2017 - 03/06/2018):
	Nodes [29782] 
	5 crashes
	0 ped injuries
	1 bikeinjuries
	0 mvo injuries
	2 right angle crashes
Master 33880 (11/23/2017 - 07/30/2018):
	Nodes [33880] 
	5 crashes
	1 ped injuries
	0 bikeinjuries
	3 mvo injuries
	2 right angle crashes
Master 33883 (07/29/

Master 40207 (04/14/2017 - 03/10/2018):
	Nodes [40207] 
	6 crashes
	1 ped injuries
	0 bikeinjuries
	0 mvo injuries
	1 right angle crashes
Master 36112 (02/23/2017 - 08/19/2017):
	Nodes [36112] 
	5 crashes
	0 ped injuries
	0 bikeinjuries
	5 mvo injuries
	5 right angle crashes
Master 52502 (08/04/2017 - 06/23/2018):
	Nodes [52502] 
	6 crashes
	1 ped injuries
	0 bikeinjuries
	5 mvo injuries
	2 right angle crashes
Master 36119 (09/07/2017 - 05/03/2018):
	Nodes [36119] 
	5 crashes
	0 ped injuries
	0 bikeinjuries
	4 mvo injuries
	4 right angle crashes
Master 36120 (04/01/2017 - 02/11/2018):
	Nodes [36120] 
	7 crashes
	0 ped injuries
	0 bikeinjuries
	6 mvo injuries
	4 right angle crashes
Master 27931 (08/29/2017 - 06/15/2018):
	Nodes [27931] 
	7 crashes
	0 ped injuries
	0 bikeinjuries
	11 mvo injuries
	4 right angle crashes
Master 58652 (03/28/2017 - 02/10/2018):
	Nodes [58652, 86185, 86284] 
	5 crashes
	0 ped injuries
	0 bikeinjuries
	4 mvo injuries
	2 right angle crashes
Master 19746 (04/10

Master 36333 (08/20/2017 - 04/19/2018):
	Nodes [36333] 
	6 crashes
	0 ped injuries
	0 bikeinjuries
	5 mvo injuries
	5 right angle crashes
Master 34287 (07/09/2017 - 04/10/2018):
	Nodes [34287] 
	5 crashes
	0 ped injuries
	0 bikeinjuries
	3 mvo injuries
	5 right angle crashes
Master 36339 (04/21/2018 - 08/06/2018):
	Nodes [36339] 
	6 crashes
	0 ped injuries
	0 bikeinjuries
	9 mvo injuries
	5 right angle crashes
Master 48628 (02/22/2017 - 11/07/2017):
	Nodes [48628] 
	5 crashes
	3 ped injuries
	1 bikeinjuries
	0 mvo injuries
	0 right angle crashes
Master 34293 (12/21/2016 - 10/24/2017):
	Nodes [34293] 
	5 crashes
	0 ped injuries
	1 bikeinjuries
	2 mvo injuries
	1 right angle crashes
Master 22016 (02/10/2018 - 07/23/2018):
	Nodes [22016] 
	7 crashes
	0 ped injuries
	0 bikeinjuries
	3 mvo injuries
	6 right angle crashes
Master 50700 (08/24/2016 - 07/02/2017):
	Nodes [50700] 
	6 crashes
	0 ped injuries
	1 bikeinjuries
	4 mvo injuries
	3 right angle crashes
Master 22031 (11/09/2017 - 05/09/2

Master 42787 (04/09/2017 - 03/23/2018):
	Nodes [42787, 84487, 98819] 
	5 crashes
	3 ped injuries
	0 bikeinjuries
	0 mvo injuries
	2 right angle crashes
Master 22313 (03/29/2017 - 11/30/2017):
	Nodes [22313] 
	5 crashes
	1 ped injuries
	1 bikeinjuries
	8 mvo injuries
	3 right angle crashes
Master 18222 (05/18/2017 - 03/22/2018):
	Nodes [18222] 
	6 crashes
	2 ped injuries
	0 bikeinjuries
	2 mvo injuries
	4 right angle crashes
Master 46897 (07/26/2017 - 06/10/2018):
	Nodes [46897] 
	6 crashes
	0 ped injuries
	0 bikeinjuries
	6 mvo injuries
	4 right angle crashes
Master 40756 (07/28/2017 - 07/19/2018):
	Nodes [40756] 
	9 crashes
	1 ped injuries
	0 bikeinjuries
	3 mvo injuries
	8 right angle crashes
Master 26424 (01/18/2017 - 12/23/2017):
	Nodes [26424] 
	5 crashes
	1 ped injuries
	0 bikeinjuries
	8 mvo injuries
	3 right angle crashes
Master 14139 (09/22/2016 - 06/07/2017):
	Nodes [14139] 
	5 crashes
	0 ped injuries
	1 bikeinjuries
	2 mvo injuries
	2 right angle crashes
Master 108348 (08/06

In [32]:
for i in range(5):
    print headers[]


SyntaxError: invalid syntax (<ipython-input-32-e67a03377f2b>, line 2)

In [48]:
print headers[1:14]

['NodeID', 'Crashes', 'Ped_Injs', 'Bike_Injs', 'MVO_Injs', 'Total_Injs', 'Right_Angle_Crashes', 'Earliest_Date', 'Latest_Date', 'Street1', 'Street2', 'X', 'Y']


In [56]:
print approved_masters.keys()

[26626, 18439, 18440, 32786, 20500, 36885, 36888, 55321, 59422, 36896, 26657, 32802, 45097, 28718, 34864, 28723, 36918, 48137, 34872, 18494, 34880, 16449, 55363, 26708, 34903, 57443, 16488, 10354, 48147, 20596, 55413, 8313, 53375, 34951, 34954, 29720, 104473, 27332, 18587, 69788, 65705, 37034, 53423, 39092, 53430, 28862, 40310, 35014, 55495, 6178, 53464, 26851, 28901, 28904, 39148, 28909, 28910, 57588, 35066, 24827, 51454, 24839, 65802, 67318, 10516, 49436, 20768, 53537, 37160, 59689, 54430, 45365, 37174, 41276, 14656, 34870, 39238, 53575, 39241, 35150, 26959, 39249, 11662, 39254, 39264, 6500, 53605, 26982, 55663, 41333, 53626, 46181, 29056, 35201, 47490, 14724, 27632, 39314, 39321, 14744, 16793, 14752, 78919, 55726, 47859, 25013, 41399, 53688, 42399, 41404, 41408, 27077, 27078, 41478, 47564, 55759, 20944, 41430, 35303, 35306, 31211, 39406, 31215, 27123, 41464, 23035, 37372, 41470, 31232, 2561, 41475, 29190, 27143, 39432, 40024, 53783, 12830, 12832, 12834, 31270, 61667, 12841, 29228, 2

In [61]:
print approved_masters[26626]

{u'0691800704': <__main__.Crash instance at 0x0BB9C670>, u'0691701209': <__main__.Crash instance at 0x0BB92C60>, u'0691800316': <__main__.Crash instance at 0x0CE84F30>, u'0691800518': <__main__.Crash instance at 0x0CE890F8>, u'0691702075': <__main__.Crash instance at 0x0BB97FA8>, u'0691701919': <__main__.Crash instance at 0x0BB92300>}
